In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format('parquet').option('header','true').option('path','abfss://bronze@adls5485.dfs.core.windows.net/customers').load()

In [0]:
df.display()

In [0]:
df = df.withColumn('domain',split(col('email'),'@')[1])
df.display()

In [0]:
df.groupBy('domain').agg(count('customer_id').alias('total_customers')).sort(col('total_customers').desc()).display()


In [0]:
df = df.withColumn('Full_Name',concat(col('first_name'),lit(' '),col("last_name")))
df = df.drop('first_name', 'last_name')
df.display()


In [0]:
df.write.format('delta').mode('overwrite').option('header','true').option('path','abfss://silver@adls5485.dfs.core.windows.net/customers').save()

In [0]:
%sql

create table if not exists sales_prj.gold.customers
using delta
location 'abfss://silver@adls5485.dfs.core.windows.net/customers'